In [3]:
%load_ext autoreload
%autoreload 2

from clonr import llms, templates

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
example = templates.SummaryExample(
    passage="""Hi all! Backstory: I just recently came out of my "ugly duckling with extreme social anxiety" phase I have never had a boyfriend/girlfriend (am also a "supervirgin") or many friends. I have really come out of my shell these past few months! I [19f] have started a new summer job and will be transferring to a university that 4 hours away. I will be quiting exactly a month from now. I met this [21m] guy at my job. He's great and we are both interested in each other. Next time I see him at work will be Saturday and his close friend told me that he is planning to ask me out. I have never had a crush like this before and this is also the first guy who is interested in me in a non-sexual context. My only issue is that he isn't in school and therefore lives permanently up by our job. I am moving into an apartment by my university and will be living down there until I graduate in 3 years (I will be an entering sophmore). I see myself as having 3 options: -We date and do it long distance -However, I am worried that I will be a bad girlfriend. My current class schedule is hectic and I still have to work to pay for my apartment. My major also requires a lot of outside of class work. This would leave very little time to talk and come up on the weekends for visits. -We have a summer fling. -When I asked my mom for advice she suggested this. According to her, going on a date will help take away some of my social anxiety around dating and make me more confident for future dates with other people. -I stop everything in its tracks now. I'm not sure what to do. And would really appreciate anyone's advice.""",
    summary="I like this guy but am moving away, should we do it long distance, have a summer fling or let it fizzle?"
)
examples = [example]

passage= """This is me browsing reddit at work again, as most of us do. This is, and has always been common practice to all of us dweller of the cube farms. I'm the stereotypical guy who works for a big corporation that doesn't give a shit about you, in turn, causing you not to give a shit about it. My predicament is as follows. **I want to care.** I used to like my job, but recently there have alot of changes as most companies come across. Jobs become more specialized, you have responsibility taken away from you and given to the most specific departments that do just that one thing. Now lets keep one thing in mind here... im not complaining about working, i know i am very fortunate to even have a job. i just feel that my lack of commitment will eventually lead to my demise, being fired. Now, i went from a fun, technical, ever changing job, to a stale, copy&paste reporting monkey job in just a year. Its become exactly like on office space where he explains a given work week, filled with simply spacing out and barely doing anything. The tasks i do have are never ending, treadmill type tasks, so at the end of the week accomplishments reports are filled with the same 6 bullets followed by "Onging". What are some things you guys do to help yourselves through the day, to be productive and feel like you contribute (if there is anything). I don't expect this thread to actually go anywhere, but i feel like i need help from those i relate to the most."""

llm = llms.LlamaCpp()
llm = llms.OpenAI()

# examples = [
#     templates.SummaryExample(passage=x.passage[:10], summary=x.summary[:10]) 
#     for x in examples
# ]

prompt = templates.Summarize.render(
    llm=llm, 
    system_prompt="You are a helpful AI assistant.",
    examples=examples,
    passage=passage
)
messages = llm.prompt_to_messages(prompt)
print(prompt)

<|im_start|>system
You are a helpful AI assistant.<|im_end|>

<|im_start|>user
Given a passage of text, write a summary of the passage. Include key points, important details, and salient information. Do not write anything other than the summary.<|im_end|>
<|im_start|>user

--- Example 1 ---<|im_end|>
<|im_start|>user
Passage: Hi all! Backstory: I just recently came out of my "ugly duckling with extreme social anxiety" phase I have never had a boyfriend/girlfriend (am also a "supervirgin") or many friends. I have really come out of my shell these past few months! I [19f] have started a new summer job and will be transferring to a university that 4 hours away. I will be quiting exactly a month from now. I met this [21m] guy at my job. He's great and we are both interested in each other. Next time I see him at work will be Saturday and his close friend told me that he is planning to ask me out. I have never had a crush like this before and this is also the first guy who is interested in m

In [10]:
import guidance

llm = guidance.llms.Mock('response')
program = guidance(templates.Summarize.guidance_render())
program(
    llm=llm, 
    system_prompt="You are a helpful AI assistant.",
    passage=
)

Stop program {{#sytem~}} 
 {{ system_prompt }} 
 {{~/system}} 


 {{#user~}} 
Given a passage of text, write a summary of the passage. Include key points, important details, and salient information. Do not write anything other than the summary. {{~#if examples}} {{/user}} {{/if}} 


 {{~#each examples}} 
 {{#user~}} 
Passage: {{this.passage}} 
 {{~/user}} 

 {{#assistant~}} 
Summary: {{this.summary}} 
 {{~/assistant}} 
 {{~/each}} 
 {{~#if examples}} {{#user}} {{/if}} 


--- Task ---
Passage: {{ passage }} 
 {{~/user}} 

 {{#assistant~}} 
Summary: {{gen "summary" temperature=0.7 max_tokens=256}} 
 {{~/assistant}}

In [80]:
import time

start = time.time()

tokens = llm.num_tokens(prompt)
print("Input tokens:", tokens)
params = llms.OpenAIGenerationParams(max_tokens=128, temperature=0.7, top_p=0.95)
r = await llm.notebook_stream(prompt, params=params)
# print("Completion tokens:", len(r))
# print("Total tokens:", tokens + len(r))
# t = time.time() - start
# print("Generation time:", round(t, 2), "seconds")
# print((tokens + len(r)) / t, "tokens / second.")
r

 I'm browsing reddit at work because I don't like my job. I'd like to
think that I have more options than just complaining. I do enjoy
giving people compliments and positive feedback though, so if you are
looking for an answer for that, I'd be happy to give it.



Response(
    completion= I'm browsing reddit + 141 chars ... , 
    time=4.65s, 
    speed=197.34 tokens/second, 
    completion_tokens=65, 
    input_tokens=852, 
    total_tokens=917
)

In [42]:
from clonr.llms.openai import OpenAIMessage
from pydantic import BaseModel

class SystemPrompt(BaseModel):
    prompt: str
    start: str
    end: str = ''

    @classmethod
    def from_llm(cls, prompt: str, llm: llms.LLM):
        return cls(prompt=prompt, start=llm.system_start, end=llm.system_end)

    def to_message(self):
        return OpenAIMessage(role='system', content=self.prompt)
    
    def __str__(self):
        return f"{self.start}{self.prompt}{self.end}"
    
s = SystemPrompt.from_llm(prompt='foo bar', llm=llms.LlamaCpp())
print(str(s))
s

### System:
foo bar



SystemPrompt(prompt='foo bar', start='### System:\n', end='\n')

In [32]:
llm.prompt_to_messages(prompt=prompt)

[OpenAIMessage(role=<RoleEnum.system: 'system'>, content='You are a helpful AI assistant.'),
 OpenAIMessage(role=<RoleEnum.user: 'user'>, content='Given a passage of text, write a summary of the passage. Include key points, important details, and salient information. Do not write anything other than the summary.'),
 OpenAIMessage(role=<RoleEnum.user: 'user'>, content='Passage: Hi all! Backstory: I just recently came out of my "ugly duckling with extreme social anxiety" phase I have never had a boyfriend/girlfriend (am also a "supervirgin") or many friends. I have really come out of my shell these past few months! I [19f] have started a new summer job and will be transferring to a university that 4 hours away. I will be quiting exactly a month from now. I met this [21m] guy at my job. He\'s great and we are both interested in each other. Next time I see him at work will be Saturday and his close friend told me that he is planning to ask me out. I have never had a crush like this before 

 A man who is struggling to find meaning and motivation at work in a
big company shares his struggles and asks for advice on how to
improve.


In [12]:
911 / 25.1

36.29482071713147

In [11]:
llm.num_tokens(prompt) + len(r)

911

In [7]:
import requests

requests.post('http://localhost:6000/v1/tokens', json=dict(prompt='hello world'))

<Response [200]>

In [11]:


llm = LlamaCpp()

# print('--- BasicChat ---')
# user_prompt = 'Recite the declaration of independence, exactly, word for word.'
# system_prompt = None
# prompt = BasicChat.render(user_prompt=user_prompt, llm=llm, system_prompt=system_prompt)
# print(prompt)

print('--- Summarize w/ context ---')
examples = [
    SummaryExampleWithContext(
        passage='Because of the high energies needed to observe tightly coiled vestigial dimensions, we must seek alternative observables in our theories', 
        summary='String theory needs new ideas since it\'s hard to verify.', 
        context='String theory has many interesting features, such as the prediction of several additional dimensions'
    )
]
prompt = SummarizeWithContext.render(
    llm=llm, 
    system_prompt="You are a helpful AI assistant.",
    context='The text is describing a geo-local phenomenon in the pacific northwest U.S., whereby meatballs fall from the sky',
    passage="Typical downpours last anywhere from 1-2 weeks, and tend to be torrential in nature. The busy season is during july and august. For centures local residents have used this debris to feed themselves during the harsh summer months. The debris is quite rich in protein.",
    examples=examples
)
llm.prompt_to_messages(prompt)
print(prompt)

--- Summarize w/ context ---
### System:
You are a helpful AI assistant.

### User:
Given a passage of text and a paragraph of context for that passage, write a summary of the passage. Include key points, important details, and make use of the context where necessary. Do not write anything other than the summary.

--- Example 1 ---
Context: String theory has many interesting features, such as the prediction of several additional dimensions
Passage: Because of the high energies needed to observe tightly coiled vestigial dimensions, we must seek alternative observables in our theories
Summary: String theory needs new ideas since it's hard to verify.


### Assistant:
Context: The text is describing a geo-local phenomenon in the pacific northwest U.S., whereby meatballs fall from the sky
Passage: Typical downpours last anywhere from 1-2 weeks, and tend to be torrential in nature. The busy season is during july and august. For centures local residents have used this debris to feed themselve

In [37]:
params = OpenAIGenerationParams(max_tokens=64, temperature=0.7)
_ = await llm.notebook_stream(prompt_or_messages=prompt, params=params)

 Meatballs fall out of the sky in certain locations during specific
times of year. Residents use them for food during harsh weather.


In [46]:
import guidance

program = guidance('''
{{#assistant~}}
Hello{{gen "world" max_tokens=1}}
{{~/assistant}}
''')

program(llm=guidance.llms.OpenAI('gpt-3.5-turbo'))

Stop program 
 {{#assistant~}} 
Hello {{gen "world" max_tokens=2}} 
 {{~/assistant}}

In [ ]:
r, text = 

Stop program Tweak this proverb to apply to model instructions instead.

 Where there is no guidance, a people falls,
but in an abundance of counselors there is safety. 
- Proverbs 11 : 14 

UPDATED
Where there is no guidance , a people falls,
but in an abundance of counselors there is safety. 
- GPT

AttributeError: 'NoneType' object has no attribute 'items'

In [5]:
params = OpenAIGenerationParams(max_tokens=64, temperature=0.7)
r, text = await llm.notebook_stream(
    'When in the course of human events it becomes',
    params=params
)

 necessary for a people to make a choice that will decide their fate,
the manner of selecting such a people is not subject to the same laws.
The selection may be by lot or election. The Romans chose a king by
lot and election. The selection by lot has never been adopted in
America, so the question of


In [15]:
from clonr.llms import LlamaCpp, OpenAIGenerationParams, MockLLM, OpenAI
from clonr.templates import Summarize, BasicChat, SummarizeWithContext, SummaryExampleWithContext

llm = LlamaCpp()
llm = OpenAI()

prompt = """
<|im_start|>user
Wow I can't believe the capital of canada is paris!
<|im_end|>
dfjkpljasdg
<|im_start|>assistant
No, the Paris is the capital of
<|im_end|>
"""

llm.prompt_to_messages(prompt)
# params = OpenAIGenerationParams(max_tokens=2, temperature=0.7)
# await llm.agenerate(prompt_or_messages=prompt, params=params)

[OpenAIMessage(role=<RoleEnum.user: 'user'>, content="Wow I can't believe the capital of canada is paris!"),
 OpenAIMessage(role=<RoleEnum.assistant: 'assistant'>, content='No, the Paris is the capital of')]

In [17]:
import guidance
from app.settings import settings

llm = guidance.llms.OpenAI('gpt-3.5-turbo-0613', api_key=settings.OPENAI_API_KEY)

In [25]:
program = guidance('''\
{{#user~}}
1, 2, 3
{{~/user}}
{{#assistant~}}
{{gen max_tokens=1}}
{{~/assistant}}''')

In [26]:
program(llm=llm)

Stop program user 1, 2, 3 assistant The

In [31]:
llm.usage 

Stop program {{#user~}} 
1, 2, 3
 {{~/user}} 
 {{#assistant~}} 
 {{gen max_tokens=1}} 
 {{~/assistant}}

In [7]:
llm = MockLLM()
params = OpenAIGenerationParams(max_tokens=64, temperature=0.7)
r, text = await llm.notebook_stream(
    'When in the course of human events it becomes',
    params=params
)

 ResponseToken1A ResponseToken1B ResponseToken1C


In [99]:
params = OpenAIGenerationParams(max_tokens=64, temperature=0.7)
r, text = await llm.notebook_stream(prompt, params=params)

 The meatballs are typically 8-10 inches wide Summary: Meatballs
falling from the sky are typical of the pacific northwest, apparently.
They are often seen during downpours and can be used as food during
the harsh summer months. It's best to avoid them.


In [30]:
import guidance

s = """\
{{#system~}}
Below is chat between a user and a helpful AI assistant. Write a response that appropriately completes the request.
{{~/system}}

{{#user~}}
Recite the declaration of independence, exactly, word for word.
{{~/user}}
"""

program = guidance.Program(s)
program(llm=llm).text

Stop program {{#system~}} 
Below is chat between a user and a helpful AI assistant. Write a response that appropriately completes the request.
 {{~/system}} 

 {{#user~}} 
Recite the declaration of independence, exactly, word for word.
 {{~/user}}

'{{#system~}}\nBelow is chat between a user and a helpful AI assistant. Write a response that appropriately completes the request.\n{{~/system}}\n\n{{#user~}}\nRecite the declaration of independence, exactly, word for word.\n{{~/user}}\n'

In [24]:
r, text = await llm.notebook_stream(prompt, params=OpenAIGenerationParams(temperature=0.7, max_tokens=64))

When in the Course of human events it becomes apparent that an
Assembly of Delegates chosen to run the country is not working to
protect basic rights of Americans, that being free speech, bearing
arms, and self-sovereignty; when it becomes clear to all citizens that
their representatives no longer represent them, they will have


In [17]:
import guidance

In [20]:
guidance.Program('hello world {{shit}}')(shit='shit')

Stop program hello world shit

In [ ]:
guidance()

In [12]:
s1 = r"""\
{{ llm.system_start -}}
{% if (system_prompt) -%}
{{ system_prompt -}}
{%- else -%}
{{ llm.default_system_prompt -}}
{%- endif %}
{{- llm.system_end }}\
"""

s2 = r"""\
{{ llm.user_start -}}
Recite the declaration of independence, exactly, word for word.
{{- llm.user_end }}

{{ llm.assistant_start -}}\
"""

print(f"{s1}{s2}")

\
{{ llm.system_start -}}
{% if (system_prompt) -%}
{{ system_prompt -}}
{%- else -%}
{{ llm.default_system_prompt -}}
{%- endif %}
{{- llm.system_end }}\
\
{{ llm.user_start -}}
Recite the declaration of independence, exactly, word for word.
{{- llm.user_end }}

{{ llm.assistant_start -}}\



In [8]:
list(llm.stream(prompt))

[OpenAIStreamResponse(created=1687566521, model='MockModel', choices=[OpenAIStreamChoice(delta=OpenAIStreamDelta(role='assistant', content='ResponseToken1A'), finish_reason=None, index=0)]),
 OpenAIStreamResponse(created=1687566521, model='MockModel', choices=[OpenAIStreamChoice(delta=OpenAIStreamDelta(role='assistant', content='ResponseToken1B'), finish_reason=None, index=0)]),
 OpenAIStreamResponse(created=1687566521, model='MockModel', choices=[OpenAIStreamChoice(delta=OpenAIStreamDelta(role='assistant', content='ResponseToken1C'), finish_reason=None, index=0)])]

In [10]:
print(prompt)

### System:
Below is chat between a user and a helpful AI assistant. Write a response that appropriately completes the request.

### User:
Recite the declaration of independence, exactly, word for word.

### Assistant:



In [3]:
model = OpenAI(OpenAIModelEnum.chatgpt_0613)

In [6]:
from clonr.utils import HFDownloader

dl = HFDownloader()

In [7]:
dl.preview_download('TheBloke/tulu-7B-GGML', regex_filter=r'.*q4_K_M.*')

error: nothing to repeat at position 0